In [36]:
from database import Database
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from numpy.linalg import norm
import numpy as np
import re

ModuleNotFoundError: No module named 'sparse_dot_topn'

In [20]:
db = Database()
offerup_df = pd.read_sql("""SELECT * FROM offerup;""", db.get_connection())
offerup_df

,id,product_name,price,condition,post_date
0,1,Nba 2k19 Xbox one,40.0,40,2018-12-13T03:52:06.513Z
1,2,Xbox one s,190.0,100,2018-12-13T03:49:34.081Z
2,3,Xbox one s Forza 4,250.0,40,2018-12-13T03:37:44.012Z
3,4,Xbox one s & 27 games $250,250.0,40,2018-12-13T03:27:02.937Z
4,5,Xbox One S 1TB Bundle For Cheap!,249.0,40,2018-12-13T03:10:35.22Z
5,6,Xbox One bundle,500.0,40,2018-12-13T03:08:21.502Z
6,7,"2TB xBox One S Gears Of War 4 Edition, Cords, ...",290.0,40,2018-12-13T03:02:25.721Z
7,8,WW2 XBOX ONE,19.0,40,2018-12-13T02:56:21.588Z
8,9,XBOX ONE X 1TB BRAND NEW 👌🎁 NEW ⭐NEW⭐NEW⭐NEW⭐NEW⭐,399.0,40,2018-12-13T02:52:38.932Z
9,10,Xbox One Day One Edition,180.0,40,2018-12-13T02:50:35.943Z


Removing emojis from names..

In [21]:
example_emojis = offerup_df['product_name'].iloc[8]
print(example_emojis)
re.sub(r'[^A-Za-z0-9_$ ]+', '', example_emojis)

XBOX ONE X 1TB BRAND NEW 👌🎁 NEW ⭐NEW⭐NEW⭐NEW⭐NEW⭐


'XBOX ONE X 1TB BRAND NEW  NEW NEWNEWNEWNEW'

In [22]:
offerup_df['product_name'] = offerup_df['product_name'].str.replace(r'[^A-Za-z0-9_$ ]+', '')
offerup_df

,id,product_name,price,condition,post_date
0,1,Nba 2k19 Xbox one,40.0,40,2018-12-13T03:52:06.513Z
1,2,Xbox one s,190.0,100,2018-12-13T03:49:34.081Z
2,3,Xbox one s Forza 4,250.0,40,2018-12-13T03:37:44.012Z
3,4,Xbox one s 27 games $250,250.0,40,2018-12-13T03:27:02.937Z
4,5,Xbox One S 1TB Bundle For Cheap,249.0,40,2018-12-13T03:10:35.22Z
5,6,Xbox One bundle,500.0,40,2018-12-13T03:08:21.502Z
6,7,2TB xBox One S Gears Of War 4 Edition Cords Wi...,290.0,40,2018-12-13T03:02:25.721Z
7,8,WW2 XBOX ONE,19.0,40,2018-12-13T02:56:21.588Z
8,9,XBOX ONE X 1TB BRAND NEW NEW NEWNEWNEWNEW,399.0,40,2018-12-13T02:52:38.932Z
9,10,Xbox One Day One Edition,180.0,40,2018-12-13T02:50:35.943Z


# N-Gram Approach

First, I will begin algorithm exploration using an n-gram approach. I will attempt to compare n-sizes of 3, 4, and 5.

In [29]:
def ngrams(b, n=3):
    # Default size is 3']
    return [b[i:i+n] for i in range(len(b)-n+1)]

In [30]:
product1 = offerup_df['product_name'].iloc[0]
ngrams(product1)

['Nba',
 'ba ',
 'a 2',
 ' 2k',
 '2k1',
 'k19',
 '19 ',
 '9 X',
 ' Xb',
 'Xbo',
 'box',
 'ox ',
 'x o',
 ' on',
 'one']

In [32]:
product_names = offerup_df['product_name']
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(product_names)

In [33]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [34]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [35]:
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.8)
matches_df = get_matches_df(matches, product_name, top=100000)
matches_df = matches_df[matches_df['similairity'] < 0.99999] # Remove all exact matches
matches_df.sample(20)

NameError: name 'ct' is not defined